### RAG Pipelines - Data Ingestion to Vector DB Pipeline

In [1]:
import langchain
import os
from langchain_community.document_loaders import TextLoader, DirectoryLoader, PyPDFLoader, PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pathlib import Path

e:\RAG-project\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
print(langchain.__version__)

1.0.0


In [3]:
### Read all the pdf
def process_all_pdfs(pdf_directory):
    all_documents = []
    pdf_dir = Path(pdf_directory)

    pdf_files = list(pdf_dir.glob("**/*.pdf"))

    print(f"Found {len(pdf_files)} PDF files.")

    for pdf_file in pdf_files:
        print(f"Processing file: {pdf_file}")

        try:
            loader = PyMuPDFLoader(str(pdf_file))
            documents = loader.load()

            for doc in documents:
                doc.metadata["source_file"] = str(pdf_file)
                doc.metadata["file_type"]='pdf'
            all_documents.extend(documents)
            print(f"Loaded {len(documents)} pages from {pdf_file}")

        except Exception as e:
            print(f"Error loading {pdf_file}: {e}")
    
    print(f"Total documents loaded: {len(all_documents)}")
    return all_documents

all_pdf_documents = process_all_pdfs('../data')

Found 2 PDF files.
Processing file: ..\data\pdf_files\sample_machine_learning (1).pdf
Loaded 1 pages from ..\data\pdf_files\sample_machine_learning (1).pdf
Processing file: ..\data\pdf_files\sample_transformers.pdf
Loaded 1 pages from ..\data\pdf_files\sample_transformers.pdf
Total documents loaded: 2


In [4]:
all_pdf_documents[1]

Document(metadata={'producer': '', 'creator': '', 'creationdate': '2025-10-18T18:43:18+00:00', 'source': '..\\data\\pdf_files\\sample_transformers.pdf', 'file_path': '..\\data\\pdf_files\\sample_transformers.pdf', 'total_pages': 1, 'format': 'PDF 1.3', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': 'D:20251018184318Z', 'page': 0, 'source_file': '..\\data\\pdf_files\\sample_transformers.pdf', 'file_type': 'pdf'}, page_content='Introduction to Transformers in AI\nTransformers are a type of deep learning model introduced in the paper "Attention is All You Need"\nin 2017. \nThey have revolutionized the field of Natural Language Processing by enabling models to handle\nsequential data without relying on recurrent architectures.\nThe key innovation of transformers is the self-attention mechanism, which allows the model to weigh\nthe importance of different words in a sequence when making predictions. \nThis capability ma

In [5]:
### Text splitting get into chunks

def split_documents(documents,chunk_size=1000,chunk_overlap=200):
    """Split documents into smaller chunks for better RAG performance"""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", " ", ""]
    )
    split_docs = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(split_docs)} chunks")
    
    # Show example of a chunk
    if split_docs:
        print(f"\nExample chunk:")
        print(f"Content: {split_docs[0].page_content[:200]}...")
        print(f"Metadata: {split_docs[0].metadata}")
    
    return split_docs

In [6]:
chunks=split_documents(all_pdf_documents)
chunks

Split 2 documents into 3 chunks

Example chunk:
Content: Introduction to Machine Learning
Machine Learning (ML) is a subset of Artificial Intelligence that enables computers to learn patterns
and make decisions without being explicitly programmed. 
Instead ...
Metadata: {'producer': '', 'creator': '', 'creationdate': '2025-10-18T18:28:51+00:00', 'source': '..\\data\\pdf_files\\sample_machine_learning (1).pdf', 'file_path': '..\\data\\pdf_files\\sample_machine_learning (1).pdf', 'total_pages': 1, 'format': 'PDF 1.3', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': 'D:20251018182851Z', 'page': 0, 'source_file': '..\\data\\pdf_files\\sample_machine_learning (1).pdf', 'file_type': 'pdf'}


[Document(metadata={'producer': '', 'creator': '', 'creationdate': '2025-10-18T18:28:51+00:00', 'source': '..\\data\\pdf_files\\sample_machine_learning (1).pdf', 'file_path': '..\\data\\pdf_files\\sample_machine_learning (1).pdf', 'total_pages': 1, 'format': 'PDF 1.3', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': 'D:20251018182851Z', 'page': 0, 'source_file': '..\\data\\pdf_files\\sample_machine_learning (1).pdf', 'file_type': 'pdf'}, page_content='Introduction to Machine Learning\nMachine Learning (ML) is a subset of Artificial Intelligence that enables computers to learn patterns\nand make decisions without being explicitly programmed. \nInstead of following fixed rules, ML systems analyze historical data to find trends and relationships,\nallowing them to make predictions or classifications on new data.\nCommon types of machine learning include supervised learning, unsupervised learning, and\nreinforcement lea

### Text Embedding and Vector DB

In [7]:
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List, Dict, Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity

In [8]:
class EmbeddingManager:
    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
        self.model_name = model_name
        self.model = None
        self._load_model()

    def _load_model(self):
        try:
            print(f"Loading embedding model: {self.model_name} ")
            self.model = SentenceTransformer(self.model_name)
            print(f"Model loaded successfully: {self.model.get_sentence_embedding_dimension()} dimensions")
        except exception as e:
            print(f"Error loading model {self.model_name}: {e}")
            raise e

    def generate_embeddings(self, texts: List[str]) -> np.ndarray:
        """
        Generate embeddings for a list of texts
        
        Args:
            texts: List of text strings to embed
            
        Returns:
            numpy array of embeddings with shape (len(texts), embedding_dim)
        """
        if not self.model:
            raise ValueError("Model not loaded")
        
        print(f"Generating embeddings for {len(texts)} texts...")
        embeddings = self.model.encode(texts, show_progress_bar=True)
        print(f"Generated embeddings with shape: {embeddings.shape}")
        return embeddings

embedding_manager = EmbeddingManager()
embedding_manager


Loading embedding model: all-MiniLM-L6-v2 
Model loaded successfully: 384 dimensions


In [9]:
import numpy as np
class VectorStore:
    """Manages document embeddings in a ChromaDB vector store"""
    
    def __init__(self, collection_name: str = "pdf_documents", persist_directory: str = "../data/vector_store"):
        """
        Initialize the vector store
        
        Args:
            collection_name: Name of the ChromaDB collection
            persist_directory: Directory to persist the vector store
        """
        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection = None
        self._initialize_store()

    def _initialize_store(self):
        """Initialize ChromaDB client and collection"""
        try:
            # Create persistent ChromaDB client
            os.makedirs(self.persist_directory, exist_ok=True)
            self.client = chromadb.PersistentClient(path=self.persist_directory)
            
            # Get or create collection
            self.collection = self.client.get_or_create_collection(
                name=self.collection_name,
                metadata={"description": "PDF document embeddings for RAG"}
            )
            print(f"Vector store initialized. Collection: {self.collection_name}")
            print(f"Existing documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error initializing vector store: {e}")
            raise

    def add_documents(self, documents, embeddings: np.ndarray):
        """
        Add documents and their embeddings to the vector store
        
        Args:
            documents: List of LangChain documents
            embeddings: Corresponding embeddings for the documents
        """
        if len(documents) != len(embeddings):
            raise ValueError("Number of documents must match number of embeddings")
        
        print(f"Adding {len(documents)} documents to vector store...")
        
        # Prepare data for ChromaDB
        ids = []
        metadatas = []
        documents_text = []
        embeddings_list = []
        
        for i, (doc, embedding) in enumerate(zip(documents, embeddings)):
            # Generate unique ID
            doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)
            
            # Prepare metadata
            metadata = dict(doc.metadata)
            metadata['doc_index'] = i
            metadata['content_length'] = len(doc.page_content)
            metadatas.append(metadata)
            
            # Document content
            documents_text.append(doc.page_content)
            
            # Embedding
            embeddings_list.append(embedding.tolist())
        
        # Add to collection
        try:
            self.collection.add(
                ids=ids,
                embeddings=embeddings_list,
                metadatas=metadatas,
                documents=documents_text
            )
            print(f"Successfully added {len(documents)} documents to vector store")
            print(f"Total documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error adding documents to vector store: {e}")
            raise

vectorstore=VectorStore()
vectorstore

Vector store initialized. Collection: pdf_documents
Existing documents in collection: 3


In [10]:
### Convert the text to embeddings

texts = [doc.page_content for doc in chunks]

### Generate embeddings
embeddings = embedding_manager.generate_embeddings(texts)

### store in the vector database
vectorstore.add_documents(chunks, embeddings)

Generating embeddings for 3 texts...


Batches: 100%|██████████| 1/1 [00:03<00:00,  3.15s/it]


Generated embeddings with shape: (3, 384)
Adding 3 documents to vector store...
Successfully added 3 documents to vector store
Total documents in collection: 6


### RAG RETRIEVER


In [11]:
class RAGRetriever:
    """Handles query-based retrieval from the vector store"""
    
    def __init__(self, vector_store: VectorStore, embedding_manager: EmbeddingManager):
        """
        Initialize the retriever
        
        Args:
            vector_store: Vector store containing document embeddings
            embedding_manager: Manager for generating query embeddings
        """
        self.vector_store = vector_store
        self.embedding_manager = embedding_manager

    def retrieve(self, query: str, top_k: int = 5, score_threshold: float = 0.0) -> List[Dict[str, Any]]:
        """
        Retrieve relevant documents for a query
        
        Args:
            query: The search query
            top_k: Number of top results to return
            score_threshold: Minimum similarity score threshold
            
        Returns:
            List of dictionaries containing retrieved documents and metadata
        """
        print(f"Retrieving documents for query: '{query}'")
        print(f"Top K: {top_k}, Score threshold: {score_threshold}")
        
        # Generate query embedding
        query_embedding = self.embedding_manager.generate_embeddings([query])[0]
        
        # Search in vector store
        try:
            results = self.vector_store.collection.query(
                query_embeddings=[query_embedding.tolist()],
                n_results=top_k
            )
            
            # Process results
            retrieved_docs = []
            
            if results['documents'] and results['documents'][0]:
                documents = results['documents'][0]
                metadatas = results['metadatas'][0]
                distances = results['distances'][0]
                ids = results['ids'][0]
                
                for i, (doc_id, document, metadata, distance) in enumerate(zip(ids, documents, metadatas, distances)):
                    # Convert distance to similarity score (ChromaDB uses cosine distance)
                    similarity_score = 1 - distance
                    
                    if similarity_score >= score_threshold:
                        retrieved_docs.append({
                            'id': doc_id,
                            'content': document,
                            'metadata': metadata,
                            'similarity_score': similarity_score,
                            'distance': distance,
                            'rank': i + 1
                        })
                
                print(f"Retrieved {len(retrieved_docs)} documents (after filtering)")
            else:
                print("No documents found")
            
            return retrieved_docs
            
        except Exception as e:
            print(f"Error during retrieval: {e}")
            return []

rag_retriever=RAGRetriever(vectorstore,embedding_manager)

In [12]:
rag_retriever.retrieve("Machine Learning")

Retrieving documents for query: 'Machine Learning'
Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 53.05it/s]

Generated embeddings with shape: (1, 384)
Retrieved 2 documents (after filtering)


[{'id': 'doc_1c95cd68_0',
  'content': 'Introduction to Machine Learning\nMachine Learning (ML) is a subset of Artificial Intelligence that enables computers to learn patterns\nand make decisions without being explicitly programmed. \nInstead of following fixed rules, ML systems analyze historical data to find trends and relationships,\nallowing them to make predictions or classifications on new data.\nCommon types of machine learning include supervised learning, unsupervised learning, and\nreinforcement learning. \nSupervised learning relies on labeled datasets to train predictive models, while unsupervised\nlearning discovers hidden structures within unlabeled data. \nReinforcement learning involves agents learning optimal behavior through trial and error interactions\nwith their environment.\nMachine learning has applications across industries, from fraud detection and recommendation\nsystems to speech recognition and self-driving cars. \nWith the growing availability of data and co

### Integration VectorDB Context pipeline with LLM output


In [16]:
### Simple RAG Pipeline with GROQ
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")

llm = ChatGroq(api_key=groq_api_key, model_name="openai/gpt-oss-20b", temperature=0.1, max_tokens=1024)


### Simple RAG function: retrieve context + genrate response

def rag_simple(query,retriever,llm, top_k=5):

    results = retriever.retrieve(query, top_k=top_k)
    context = "\n\n".join([doc['content'] for doc in results]) if results else ""
    print("Context",context)
    if not context:
        return "No relevant context found."

    ## generate the answwer using GROQ LLM
    prompt=f"""Use the following context to answer the question concisely.
        Context:
        {context}

        Question: {query}

        Answer:"""
    
    response=llm.invoke([prompt.format(context=context,query=query)])
    return response.content
    

In [18]:
answer=rag_simple("What is ML?",rag_retriever,llm)
print(answer)

Retrieving documents for query: 'What is ML?'
Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 83.32it/s]

Generated embeddings with shape: (1, 384)
Retrieved 2 documents (after filtering)
Context systems to speech recognition and self-driving cars. 
With the growing availability of data and computing power, ML is becoming an essential tool for
solving complex problems in both academia and industry.

systems to speech recognition and self-driving cars. 
With the growing availability of data and computing power, ML is becoming an essential tool for
solving complex problems in both academia and industry.


**ML** stands for **Machine Learning**—a branch of artificial intelligence that uses data and computational power to train models that can learn patterns and make predictions or decisions.


In [20]:
# --- Enhanced RAG Pipeline Features ---
def rag_advanced(query, retriever, llm, top_k=5, min_score=0.2, return_context=False):
    """
    RAG pipeline with extra features:
    - Returns answer, sources, confidence score, and optionally full context.
    """
    results = retriever.retrieve(query, top_k=top_k, score_threshold=min_score)
    if not results:
        return {'answer': 'No relevant context found.', 'sources': [], 'confidence': 0.0, 'context': ''}
    
    # Prepare context and sources
    context = "\n\n".join([doc['content'] for doc in results])
    sources = [{
        'source': doc['metadata'].get('source_file', doc['metadata'].get('source', 'unknown')),
        'page': doc['metadata'].get('page', 'unknown'),
        'score': doc['similarity_score'],
        'preview': doc['content'][:300] + '...'
    } for doc in results]
    confidence = max([doc['similarity_score'] for doc in results])
    
    # Generate answer
    prompt = f"""Use the following context to answer the question concisely.\nContext:\n{context}\n\nQuestion: {query}\n\nAnswer:"""
    response = llm.invoke([prompt.format(context=context, query=query)])
    
    output = {
        'answer': response.content,
        'sources': sources,
        'confidence': confidence
    }
    if return_context:
        output['context'] = context
    return output

# Example usage:
result = rag_advanced("ML", rag_retriever, llm, top_k=3, min_score=0.1, return_context=True)
print("Answer:", result['answer'])
print("Sources:", result['sources'])
print("Confidence:", result['confidence'])
print("Context Preview:", result['context'][:300])

Retrieving documents for query: 'ML'
Top K: 3, Score threshold: 0.1
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00,  3.03it/s]

Generated embeddings with shape: (1, 384)
Retrieved 0 documents (after filtering)
Answer: No relevant context found.
Sources: []
Confidence: 0.0
Context Preview: 


In [21]:
# --- Advanced RAG Pipeline: Streaming, Citations, History, Summarization ---
from typing import List, Dict, Any
import time

class AdvancedRAGPipeline:
    def __init__(self, retriever, llm):
        self.retriever = retriever
        self.llm = llm
        self.history = []  # Store query history

    def query(self, question: str, top_k: int = 5, min_score: float = 0.2, stream: bool = False, summarize: bool = False) -> Dict[str, Any]:
        # Retrieve relevant documents
        results = self.retriever.retrieve(question, top_k=top_k, score_threshold=min_score)
        if not results:
            answer = "No relevant context found."
            sources = []
            context = ""
        else:
            context = "\n\n".join([doc['content'] for doc in results])
            sources = [{
                'source': doc['metadata'].get('source_file', doc['metadata'].get('source', 'unknown')),
                'page': doc['metadata'].get('page', 'unknown'),
                'score': doc['similarity_score'],
                'preview': doc['content'][:120] + '...'
            } for doc in results]
            # Streaming answer simulation
            prompt = f"""Use the following context to answer the question concisely.\nContext:\n{context}\n\nQuestion: {question}\n\nAnswer:"""
            if stream:
                print("Streaming answer:")
                for i in range(0, len(prompt), 80):
                    print(prompt[i:i+80], end='', flush=True)
                    time.sleep(0.05)
                print()
            response = self.llm.invoke([prompt.format(context=context, question=question)])
            answer = response.content

        # Add citations to answer
        citations = [f"[{i+1}] {src['source']} (page {src['page']})" for i, src in enumerate(sources)]
        answer_with_citations = answer + "\n\nCitations:\n" + "\n".join(citations) if citations else answer

        # Optionally summarize answer
        summary = None
        if summarize and answer:
            summary_prompt = f"Summarize the following answer in 2 sentences:\n{answer}"
            summary_resp = self.llm.invoke([summary_prompt])
            summary = summary_resp.content

        # Store query history
        self.history.append({
            'question': question,
            'answer': answer,
            'sources': sources,
            'summary': summary
        })

        return {
            'question': question,
            'answer': answer_with_citations,
            'sources': sources,
            'summary': summary,
            'history': self.history
        }

# Example usage:
adv_rag = AdvancedRAGPipeline(rag_retriever, llm)
result = adv_rag.query("what is attention is all you need", top_k=3, min_score=0.1, stream=True, summarize=True)
print("\nFinal Answer:", result['answer'])
print("Summary:", result['summary'])
print("History:", result['history'][-1])

Retrieving documents for query: 'what is attention is all you need'
Top K: 3, Score threshold: 0.1
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 99.98it/s]

Generated embeddings with shape: (1, 384)
Retrieved 0 documents (after filtering)



Final Answer: No relevant context found.
Summary: The response indicates that no relevant context was found. Consequently, no additional information can be provided.
History: {'question': 'what is attention is all you need', 'answer': 'No relevant context found.', 'sources': [], 'summary': 'The response indicates that no relevant context was found. Consequently, no additional information can be provided.'}
